Parse OSM file and export the result in CSV

In [1]:
import xml.etree.ElementTree
import re

In [2]:
dataFileName = 'example.osm'

In [30]:
def parseElement(element):
    targetElement = ['node', 'way']
    nodeAttribs =  ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
    wayAttribs = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
    
    
    
    if element.tag in targetElement:
        print('Parsing: {}'.format(element.tag))
        if element.tag == 'node':
            selectedAttribs = nodeAttribs
        else:
            selectedAttribs = wayAttribs
        
        attribs = element.attrib
        elem = {}
        for i in selectedAttribs:
            if i in ['id', 'uid', 'changeset']:
                elem[i] = int(attribs[i])
            else:
                elem[i] = attribs[i]
            
        print(elem)
        
def parseOSMfile(fileName):
    e = xml.etree.ElementTree.parse(fileName).getroot()
    for element in e:
        parseElement(element)

In [31]:
parseOSMfile(dataFileName)

Parsing: node
{'id': 261114295, 'lat': '41.9730791', 'lon': '-87.6866303', 'user': 'bbmiller', 'uid': 451048, 'version': '7', 'changeset': 11129782, 'timestamp': '2012-03-28T18:31:23Z'}
Parsing: node
{'id': 261114296, 'lat': '41.9730416', 'lon': '-87.6878512', 'user': 'bbmiller', 'uid': 451048, 'version': '6', 'changeset': 8448766, 'timestamp': '2011-06-15T17:04:54Z'}
Parsing: node
{'id': 261114299, 'lat': '41.9729565', 'lon': '-87.6939548', 'user': 'bbmiller', 'uid': 451048, 'version': '5', 'changeset': 8581395, 'timestamp': '2011-06-29T14:14:14Z'}
Parsing: node
{'id': 261146436, 'lat': '41.9707380', 'lon': '-87.6976025', 'user': 'bbmiller', 'uid': 451048, 'version': '5', 'changeset': 8581395, 'timestamp': '2011-06-29T14:14:14Z'}
Parsing: node
{'id': 261147304, 'lat': '41.9740068', 'lon': '-87.6988576', 'user': 'bbmiller', 'uid': 451048, 'version': '7', 'changeset': 8581395, 'timestamp': '2011-06-29T14:14:15Z'}
Parsing: node
{'id': 261224274, 'lat': '41.9707656', 'lon': '-87.6938669',